# Imports and Paths

In [1]:
import os
import sys

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
if os.name == 'nt':
    PATH_base = 'E:\\GitHub\\data_science\\'
    PATH_bd = 'E:\\GitHub\\data_science\\data\\uncompressed\\blue_book_for_bulldozers\\'
    PATH_func = 'E:\\GitHub\\data_science\\src\\'
elif os.name == 'posix':
    PATH = '/home/msnow/git/data_science/'
    DATA = '/data/msnow/data_science/'

In [5]:
sys.path.append(PATH+'src/')

In [7]:
from features.imports import *
from features.utilities import *
from features.fastai import *

# Blue Book for Bulldozers

This dataset is from the kaggle competition [Blue Book for Bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers).  

In [10]:
bb_raw = pd.read_csv(f'{DATA}/bbfb/Train.csv', low_memory=False, parse_dates=["saledate"])

Save the raw dataframe

In [80]:
# bb_raw.to_feather(f'{DATA}/bbfb/bulldozer_raw')
bb_raw = pd.read_feather(f'{DATA}/bbfb/bulldozer_raw')

Display the first few rows of the dataframe

In [82]:
display_some(bb_raw.head().T,100,10)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


## Metrics

In this competition, the goal is to predict the SalePrice and the metric used is the root mean squared log error (RMSLE). Therefore, I will convert the SalePrice column into the log of the SalePrice

In [83]:
bb_raw.SalePrice = np.log(bb_raw.SalePrice)

## Splitting Datetime

Split the dattime column `saledate` into multiple columns containing integer components of the datetime, e.g., year, month, day, day of week, weekend

In [85]:
add_datepart(bb_raw, 'saledate')

## Convert Strings to Integer Categorical

Convert all columns with string values to integer categorical variables

In [87]:
train_cats(bb_raw)

In [88]:
bb_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

When there is ordinality in the strings you can assign the categorical integer values to share that order

In [89]:
df_codes = bb_raw.UsageBand.cat.codes
bb_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [90]:
bb_UsageBand_cat = pd.DataFrame({'Raw':bb_raw.UsageBand, 'Unordered':df_codes, 'Ordered':bb_raw.UsageBand.cat.codes})
bb_UsageBand_cat.head()

,Ordered,Raw,Unordered
0,2,Low,1
1,2,Low,1
2,0,High,0
3,0,High,0
4,1,Medium,2


Save the processed dataframe

In [92]:
bb_raw.to_feather(f'{DATA}/bbfb/bulldozer')
# bb_raw = pd.read_feather(f'{DATA}/bbfb/bulldozer')

Split off the response variable, and change the dataframe into an entirely numeric dataframe

In [25]:
bb, y, na_dict = proc_df(bb_raw, 'SalePrice')

# Corporación Favorita Grocery Sales Forecasting 

This dataset is from the kaggle competition [Corporación Favorita Grocery Sales Forecasting](https://www.kaggle.com/c/favorita-grocery-sales-forecasting).  

## Explore dataset before converting to dataframe

In [59]:
g_df = pd.read_csv(f'{DATA}favorita/train.csv', nrows = 10**3)

In [60]:
display_some(g_df.head().T,100,10)

,0,1,2,3,4
id,0,1,2,3,4
date,2013-01-01,2013-01-01,2013-01-01,2013-01-01,2013-01-01
store_nbr,25,25,25,25,25
item_nbr,103665,105574,105575,108079,108701
unit_sales,7,1,2,1,1
onpromotion,NaN,NaN,NaN,NaN,NaN


In [69]:
g_df = pd.read_csv(f'{DATA}favorita/train.csv', nrows = 5*10**7, usecols=['onpromotion'], low_memory=False)

/data/msnow/miniconda3/envs/cord_comp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
g_df.onpromotion.unique()

array([nan, False, True], dtype=object)

# Upload Full Dataset

In [32]:
g_types = {'id':'int64', 'store_nbr':'int8', 'item_nbr':'int32', 'unit_sales':'float32', 'onpromotion':'object'}

If you try to read it in without setting the data types you often run into an out of memory error as there are 125 million rows

In [71]:
g_raw = pd.read_csv(f'{DATA}favorita/train.csv', parse_dates=['date'], dtype=g_types, infer_datetime_format=True, nrows = 5*10**7, low_memory=False)

As per the data dictionary : 
 - The `onpromotion` column tells whether that `item_nbr` was on promotion for a specified `date` and `store_nbr`

As we saw above the actual column contains, either nans, the string True or the string False.  So we want to convert that to a boolean variable

In [72]:
g_raw.onpromotion.fillna(False, inplace=True)
g_raw.onpromotion = g_raw.onpromotion.map({'False': False, 'True':True})
g_raw.onpromotion = g_raw.onpromotion.astype(bool)

## Save to Feather

In [73]:
g_raw.to_feather(f'{DATA}favorita/groceries_raw')

## Show properties of the data

In [93]:
g_raw.describe(include='all')

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,5.000000e+07,50000000,5.000000e+07,5.000000e+07,5.000000e+07,50000000
unique,NaN,906,NaN,NaN,NaN,2
top,NaN,2015-06-20 00:00:00,NaN,NaN,NaN,False
freq,NaN,84878,NaN,NaN,NaN,27538331
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2015-06-27 00:00:00,NaN,NaN,NaN,NaN
mean,2.500000e+07,NaN,2.710937e+01,7.834337e+05,8.914280e+00,NaN
std,1.443376e+07,NaN,1.651728e+01,4.033173e+05,1.972376e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,1.250000e+07,NaN,1.100000e+01,4.177630e+05,2.000000e+00,NaN


## Repeat the process on the test data set for comparison

In [75]:
g_test = pd.read_csv(f'{DATA}favorita/test.csv',parse_dates=['date'], dtype=g_types, infer_datetime_format=True)

In [76]:
g_test.onpromotion.fillna(False, inplace=True)
g_test.onpromotion = g_test.onpromotion.map({'False': False, 'True':True})
g_test.onpromotion = g_test.onpromotion.astype(bool)

In [77]:
g_test.describe(include='all')

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,3171867
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


## Metrics

In this competition, the goal is to predict the `unit_sales` and the metric used is the Normalized Weighted Root Mean Squared Logarithmic Error  (NWRMSLE). Which is calculated as follows

$$
NWRMSLE = \sqrt{\dfrac{\sum\limits_{i=1}^nw_i\left(\ln \left(\hat{y_i} + 1 \right) - \ln \left(y_i + 1\right)\right)^2}{\sum\limits_{i=1}^n w_i}}
$$


where for row $i$, $\hat{y_i}$ is the predicted unit_sales of an item and $y_i$ is the actual unit_sales; $n$ is the total number of rows in the test set.
Therefore, I will convert the `unit_sales` column into the log of the (`unit_sales` + 1)

Also, in the compteition descriptions they say that "Negative values of unit_sales represent returns of that particular item.". which we will consider to be 0, and so we should remove them from the data before converting to log format (`np.clip`)

In [94]:
g_raw.unit_sales = np.log1p(np.clip(g_raw.unit_sales,0,None))

## Splitting Datetime

In [95]:
add_datepart(g_raw, 'date')

No need to do the Convert Strings to Integer Categorical step as all the columns are numerical to begin with

## Save to feather

In [96]:
g_raw.to_feather(f'{DATA}favorita/groceries')